## High-level interface design

In this design, the `strategy` class wraps the standard `estimator` class from scikit-learn. The task class encapsulates the data preprocessing that is needed to separate the feature data and to derive the target variable. The task object is a [visitor](https://en.wikipedia.org/wiki/Visitor_pattern) to the strategy class, adding new operations to the existent estimator object without modifying the estimator itself. In particular, it adds new operations to the `fit()` and `predict()` method before calling the estimator's `fit()` and `predict()` method. For a more detailed description, see https://github.com/kiraly-group/sktime/issues/1. 

In [62]:
from abc import ABC, abstractmethod
from sklearn.base import BaseEstimator
NOT_IMPLEMENTED = "Needs to be implemented."

        
class TSClassifier(BaseEstimator):
    def fit(self, data):
        print(f'Fitting classifier on data with {data.nrows} rows.')
        
    def predict(self, data):
        print(f'Predicting {data.nrows} rows.')


class BaseTask(ABC):
    def __init__(self, data=None, specs=None):
        self.data = data
        self.specs = specs
        print('Specifying task.')

    @abstractmethod
    def setup_fit(self):
        raise NotImplementedError(NOT_IMPLEMENTED)
        
    @abstractmethod
    def setup_predict(self):
        raise NotImplementedError(NOT_IMPLEMENTED)

        
class Task(BaseTask):
    def setup_fit(self):
        print('Setting up task for fitting.')
                
    def setup_predict(self):
        print('Setting up task for predicting.')
        
        
class BaseStrategy(ABC):
    def __init__(self, estimator):
        self.estimator = estimator
        self.task = None
    
    @abstractmethod
    def fit(self, task, data=None):
        raise NotImplementedError(NOT_IMPLEMENTED)
        
    @abstractmethod
    def predict(self, data=None):
        raise NotImplementedError(NOT_IMPLEMENTED)
    
    def set_params(self):
        self.estimator.set_params()
        
    def get_params(self):
        self.estimator.get_params()
        
        
class Strategy(BaseStrategy):
    def fit(self, task, data=None):
        self.task = task
        
        task.setup_fit()
        if data is None:
            data = task.data
        self.estimator.fit(data)
    
    def predict(self, data=None):
        if data is None:
            data = self.task.data
        self.task.setup_predict()
        self.estimator.predict(data)


class XPandas:
    def __init__(self, nrows):
        self.nrows = nrows

In [63]:
data = XPandas(nrows=100)
estimator = TSClassifier()
task = Task(data)
strategy = Strategy(estimator)     
strategy.fit(task)
strategy.predict()

Specifying task.
Setting up task for fitting.
Fitting classifier on data with 100 rows.
Setting up task for predicting.
Predicting 100 rows.


## Benchmarking interface?

In [43]:
def benchmark(task, datasets, outer_cv, strategies, metrics):
    for dataset in datasets:
        for train, test in outer_cv.split(dataset):
            for strategy in strategies:
                strategy.fit(task, train)
                y_pred, y_test = strategy.predict(task, test) 
                for metric in metrics: 
                    mean, var, stderr = metric(y_pred, y_test)

In [ ]:
data = XPandas(nrows=100)
task = Task() # without data
strategy = Strategy(estimator)
cv = Kfold()
metrics = [mae, mse]
benchmark(task, data, cv, strategy, metrics)

## Interface design with feature-target window splits

In [47]:
from abc import ABCMeta, abstractmethod
from sklearn.base import BaseEstimator

        
class TSClassifier(BaseEstimator):
    def fit(self, X, y):
        print(f'Fitting classifier on data with {data.nrows} rows.')
        
    def predict(self, X, y):
        print(f'Predicting {data.nrows} rows.')

        
class Task:
    def __init__(self, data, specs=None):
        self.data = data
        self.specs = specs
        print('Specifying task.')
    
    def setup_fit(self):
        print('Setting up task for fitting.')
        X, y = split_in_time(data)
        return X, y
    
    def setup_predict(self):
        print('Setting up task for predicting.')
        X, y = split_in_time(data)
        return X, y
    
        
NOT_IMPLEMENTED = "Needs to be implemented."
class BaseStrategy:
    __metaclass__ = ABCMeta
    @abstractmethod
    def __init__(self, estimator):
        self.estimator = estimator
        self.task = None
    
    @abstractmethod
    def fit(self, task, data=None):
        raise NotImplementedError(NOT_IMPLEMENTED)
        
    @abstractmethod
    def predict(self, data=None):
        raise NotImplementedError(NOT_IMPLEMENTED)
    
    @abstractmethod
    def set_params(self):
        self.estimator.set_params()
        
    @abstractmethod
    def get_params(self):
        self.estimator.get_params()
        
        
class Strategy(BaseStrategy):
    def fit(self, task, data=None):
        self.task = task
        
        if data is None:
            data = task.data

        X, y = task.setup_fit()
        self.estimator.fit(X, y)
    
    def predict(self, data=None):
        if data is None:
            data = self.task.data
            
        X, y = self.task.setup_predict()
        estimator.predict(X, y)


class XPandas:
    def __init__(self, nrows):
        self.nrows = nrows
        
        
def split_in_time(data, columns=None, window=None, target_test_window=False):
    feature = XPandas(nrows=data.nrows)
    target_train = XPandas(nrows=data.nrows)
    if target_test_window:
        target_test = XPandas(nrows=data.nrows)
        return feature, target_train, target_test
    return feature, target_train

In [48]:
data = XPandas(nrows=100)
estimator = TSClassifier()
task = Task(data)
strategy = Strategy(estimator)     
strategy.fit(task)
strategy.predict()

Specifying task.
Setting up task for fitting.
Fitting classifier on data with 100 rows.
Setting up task for predicting.
Predicting 100 rows.
